In [7]:
# ============================================
# Minimal Personal Task Assistant Crew (Gemini + Serper) — CLEAN + WORKING
# ============================================
!pip install -qU crewai crewai-tools google-generativeai

import os, re
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool  # Valid BaseTool for CrewAI

# 1) Keys (set ONLY GOOGLE_API_KEY to avoid warnings)
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"  # https://ai.google.dev
os.environ["SERPER_API_KEY"] = "SERPER_API_KEY"  # https://serper.dev
os.environ.pop("GEMINI_API_KEY", None)  # silence: "Both GOOGLE_API_KEY and GEMINI_API_KEY are set"

# 2) LLM (Google Gemini)
llm = LLM(model="gemini/gemini-2.5-flash", temperature=0.6, api_key=os.environ["GOOGLE_API_KEY"])

# 3) Real web search tool (Serper)
web_search = SerperDevTool()  # reads SERPER_API_KEY from env

# 4) Agents
planner = Agent(
    role="🧠 Planner",
    goal="Break a goal into clear, ordered subtasks with dependencies.",
    backstory="You create concise, actionable study/work plans.",
    allow_delegation=False,
    verbose=False,
    llm=llm
)

executor = Agent(
    role="🧑‍💻 Executor",
    goal="Use tools to assemble a concrete schedule with dates, links, and time estimates.",
    backstory="You turn plans into day-by-day schedules with practical details.",
    tools=[web_search],  # Keep it simple: real search only
    allow_delegation=False,
    verbose=False,
    llm=llm
)

# 5) Helpers
def parse_days(goal: str, default=7) -> int:
    m = re.search(r'(\d+)\s*(day|days|d)\b', goal, re.I)
    return int(m.group(1)) if m else default

def date_list(days: int) -> str:
    today = datetime.now()
    return "\n".join(
        f"Day {i+1}: {(today + timedelta(days=i)).strftime('%Y-%m-%d (%A)')}"
        for i in range(days)
    )

# 6) Tasks (Planner → Executor via context)
def planning_task(goal: str) -> Task:
    return Task(
        description=f"""
        Goal: "{goal}"
        - Break into 6–10 subtasks in logical order (beginner → advanced)
        - Note any dependencies
        - Keep each subtask completable in ~1 session
        Output a numbered list.
        """,
        expected_output="A numbered, ordered list of subtasks with brief descriptions.",
        agent=planner
    )

def execution_task(goal: str, plan_task: Task) -> Task:
    days = parse_days(goal, default=7)
    dates = date_list(days)
    return Task(
        description=f"""
        Build a complete day-by-day schedule for: "{goal}"
        Use tools:
        - Web Search (Serper): find key topics/resources (e.g., 'RHCSA syllabus', 'RHCSA labs', 'SELinux basics')
        Instructions:
        - Use these dates (one subtask per day when possible):
{dates}
        - Estimate time: theory 1–2h, hands-on 3–4h, review 1h
        - Include at least 1 actionable tip and 1–2 useful links per day when relevant
        Output a clear list: Day, Date, Topics, Estimated time, Tip, Links.
        """,
        expected_output="A day-by-day schedule with date, topics, estimates, tips, and links.",
        agent=executor,
        context=[plan_task]  # Executor receives the Planner's output
    )

# 7) Orchestration
def run_crew(goal: str):
    p_task = planning_task(goal)
    e_task = execution_task(goal, p_task)
    crew = Crew(
        agents=[planner, executor],
        tasks=[p_task, e_task],
        process=Process.sequential,
        verbose=1
    )
    print(f"\n🎯 Goal: {goal}\n")
    result = crew.kickoff()
    print("\n✅ Final Result:\n")
    print(result)
    return result

# 8) Example
result = run_crew("Plan a study schedule for RHCSA in 10 days")



🎯 Goal: Plan a study schedule for RHCSA in 10 days



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e8b81afc-02ca-4eb1-a8a2-51e21233eab8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 🧠 Planner                                                                                              │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Goal: "Plan a study schedule for RHCSA in 10 days"                                                     │
│          - Break into 6–10 subtasks in logical order (beginner → advanced)                                      │
│          - Note any dependencies                                                                                │
│          - Keep each subtask completable in ~1 session                                                          │
│          Output a numbered list.                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 🧠 Planner                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1.  **Day 1: Basic System Navigation & File Management**                                                       │
│      *   **Content:** Navigate the file system (ls, cd, pwd), manage files and directories (cp, mv, rm, mkdir,  │
│  rmdir), understand and modify file permissions (chmod, chown, chgrp), use common text utilities (cat, less,    │
│  grep, find).                                                                                                   │
│      *   **Dependencies:** None                                                                                 │
│                                                                                                                 │
│  2.  **Day 2: User & Group Management, Process Control**                                                        │
│      *   **Content:** Create, modify, and delete local user accounts and groups (useradd, usermod, userdel,     │
│  groupadd, groupmod, groupdel), manage user passwords (passwd), configure `sudo` access (`visudo`), manage      │
│  running processes (ps, top, kill, killall, bg, fg, jobs).                                                      │
│      *   **Dependencies:** Day 1 (basic command-line usage, file permissions)                                   │
│                                                                                                                 │
│  3.  **Day 3: Software Management & System Services**                                                           │
│      *   **Content:** Install, update, and remove software packages using `dnf`/`yum`, configure software       │
│  repositories, manage system services with `systemctl` (start, stop, enable, disable, status), schedule tasks   │
│  with `cron` (`crontab -e`), basic `journalctl` usage.                                                          │
│      *   **Dependencies:** Day 1 (command-line basics)                                                          │
│                                                                                                                 │
│  4.  **Day 4: Storage Management - Filesystems & Mounting**                                                     │
│      *   **Content:** Identify storage devices (`lsblk`, `fdisk -l`, `parted`), create and manage disk          │
│  partitions, create and verify filesystems (ext4, XFS), mount and unmount filesystems, configure persistent     │
│  mounts via `/etc/fstab`.                                                                                       │
│      *   **Dependencies:** Day 1 (basic file system understanding)                                              │
│                                                                                                                 │
│  5.  **Day 5: Storage Management - LVM**                                                                        │
│      *   **Content:** Create and manage Physical Volumes (PVs), Volume Groups (VGs), and Logical Volumes        │
│  (LVs), extend and reduce LVM components, mount LVM-based filesystems.                                          │
│      *   **Dependencies:** Day 4 (understanding partitions and filesystems)                                     │
│                                                                                                                 │
│  6.  **Day 6: Networking Configuration & Firewall**     

...

TypeError: Markdown expects text, not CrewOutput(raw="Day-by-Day RHCSA Study Schedule (10 Days)\n\n**Day 1: 2025-11-03 (Monday)**\n*   **Topics:** Basic System Navigation & File Management: Navigate the file system (ls, cd, pwd), manage files and directories (cp, mv, rm, mkdir, rmdir), understand and modify file permissions (chmod, chown, chgrp), use common text utilities (cat, less, grep, find).\n*   **Estimated Time:** Theory: 1.5h, Hands-on: 3.5h, Review: 1h\n*   **Tip:** Spend significant time in the terminal. Repetition builds muscle memory for these fundamental commands.\n*   **Links:**\n    *   RHCSA.GURU - Access Linux File System: https://rhcsa.guru/?name=rh124-access-linux-filesystem\n    *   DigitalOcean - Basic Linux Navigation and File Management: https://www.digitalocean.com/community/tutorials/basic-linux-navigation-and-file-management\n\n**Day 2: 2025-11-04 (Tuesday)**\n*   **Topics:** User & Group Management, Process Control: Create, modify, and delete local user accou

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 03b038b3-8ca6-430b-a245-c0cd5d69a6d4                                                                     │
│  Agent: 🧑‍💻 Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 🧑‍💻 Executor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Day-by-Day RHCSA Study Schedule (10 Days)                                                                      │
│                                                                                                                 │
│  **Day 1: 2025-11-03 (Monday)**                                                                                 │
│  *   **Topics:** Basic System Navigation & File Management: Navigate the file system (ls, cd, pwd), manage      │
│  files and directories (cp, mv, rm, mkdir, rmdir), understand and modify file permissions (chmod, chown,        │
│  chgrp), use common text utilities (cat, less, grep, find).                                                     │
│  *   **Estimated Time:** Theory: 1.5h, Hands-on: 3.5h, Review: 1h                                               │
│  *   **Tip:** Spend significant time in the terminal. Repetition builds muscle memory for these fundamental     │
│  commands.                                                                                                      │
│  *   **Links:**                                                                                                 │
│      *   RHCSA.GURU - Access Linux File System: https://rhcsa.guru/?name=rh124-access-linux-filesystem          │
│      *   DigitalOcean - Basic Linux Navigation and File Management:                                             │
│  https://www.digitalocean.com/community/tutorials/basic-linux-navigation-and-file-management                    │
│                                                                                                                 │
│  **Day 2: 2025-11-04 (Tuesday)**                                                                                │
│  *   **Topics:** User & Group Management, Process Control: Create, modify, and delete local user accounts and   │
│  groups (useradd, usermod, userdel, groupadd, groupmod, groupdel), manage user passwords (passwd), configure    │
│  `sudo` access (`visudo`), manage running processes (ps, top, kill, killall, bg, fg, jobs).                     │
│  *   **Estimated Time:** Theory: 1.5h, Hands-on: 3.5h, Review: 1h                                               │
│  *   **Tip:** Practice `sudo` configuration carefully. Incorrect `visudo` entries can lock you out. Always use  │
│  `visudo` to edit the sudoers file.                                                                             │
│  *   **Links:**                                                                                                 │
│      *   Red Hat Docs - Managing Users and Groups:                                                              │
│  https://docs.redhat.com/en/documentation/red_hat_enterprise_linux/7/html/system_administrators_guide/ch-manag  │
│  ing_users_and_groups                                                                                           │
│      *   Tecmint - How to Manage Users and Groups in RHEL (Part 5):                                             │
│  https://www.tecmint.com/manage-users-and-groups-linux/                                                         │
│                                                                                                                 │
│  **Day 3: 2025-11-05 (Wednesday)**                                                                              │
│  *   **Topics:** Software Management & System Services: 

Would you like to view your execution traces? [y/N] (20s timeout): 

In [8]:
from IPython.display import Markdown

# CrewAI returns CrewOutput — access its raw text
Markdown(result.raw)

Day-by-Day RHCSA Study Schedule (10 Days)

**Day 1: 2025-11-03 (Monday)**
*   **Topics:** Basic System Navigation & File Management: Navigate the file system (ls, cd, pwd), manage files and directories (cp, mv, rm, mkdir, rmdir), understand and modify file permissions (chmod, chown, chgrp), use common text utilities (cat, less, grep, find).
*   **Estimated Time:** Theory: 1.5h, Hands-on: 3.5h, Review: 1h
*   **Tip:** Spend significant time in the terminal. Repetition builds muscle memory for these fundamental commands.
*   **Links:**
    *   RHCSA.GURU - Access Linux File System: https://rhcsa.guru/?name=rh124-access-linux-filesystem
    *   DigitalOcean - Basic Linux Navigation and File Management: https://www.digitalocean.com/community/tutorials/basic-linux-navigation-and-file-management

**Day 2: 2025-11-04 (Tuesday)**
*   **Topics:** User & Group Management, Process Control: Create, modify, and delete local user accounts and groups (useradd, usermod, userdel, groupadd, groupmod, groupdel), manage user passwords (passwd), configure `sudo` access (`visudo`), manage running processes (ps, top, kill, killall, bg, fg, jobs).
*   **Estimated Time:** Theory: 1.5h, Hands-on: 3.5h, Review: 1h
*   **Tip:** Practice `sudo` configuration carefully. Incorrect `visudo` entries can lock you out. Always use `visudo` to edit the sudoers file.
*   **Links:**
    *   Red Hat Docs - Managing Users and Groups: https://docs.redhat.com/en/documentation/red_hat_enterprise_linux/7/html/system_administrators_guide/ch-managing_users_and_groups
    *   Tecmint - How to Manage Users and Groups in RHEL (Part 5): https://www.tecmint.com/manage-users-and-groups-linux/

**Day 3: 2025-11-05 (Wednesday)**
*   **Topics:** Software Management & System Services: Install, update, and remove software packages using `dnf`/`yum`, configure software repositories, manage system services with `systemctl` (start, stop, enable, disable, status), schedule tasks with `cron` (`crontab -e`), basic `journalctl` usage.
*   **Estimated Time:** Theory: 2h, Hands-on: 3h, Review: 1h
*   **Tip:** Understand the difference between `enable` and `start` for `systemctl`. `enable` ensures the service starts at boot, `start` only starts it for the current session.
*   **Links:**
    *   DigitalOcean - How To Use journalctl to View and Manipulate systemd Logs: https://www.digitalocean.com/community/tutorials/how-to-use-journalctl-to-view-and-manipulate-systemd-logs
    *   ADDZEY.DEV - RHCSA Study Notes (Schedule tasks, systemctl): https://addzey.dev/rhcsa/

**Day 4: 2025-11-06 (Thursday)**
*   **Topics:** Storage Management - Filesystems & Mounting: Identify storage devices (`lsblk`, `fdisk -l`, `parted`), create and manage disk partitions, create and verify filesystems (ext4, XFS), mount and unmount filesystems, configure persistent mounts via `/etc/fstab`.
*   **Estimated Time:** Theory: 1.5h, Hands-on: 4h, Review: 1h
*   **Tip:** Be extremely careful when working with `fdisk`, `parted`, and `mkfs`. Practice in a virtual machine extensively to avoid data loss.
*   **Links:**
    *   KodeKloud - Configure systems to mount file systems at or during boot: https://notes.kodekloud.com/docs/Red-Hat-Certified-System-AdministratorRHCSA/Create-and-Configure-File-Systems/Configure-systems-to-mount-file-systems-at-or-during-boot
    *   LinuxConfig.org - Mount Disks by UUID or Label on RHEL 8: https://linuxconfig.org/configure-systems-to-mount-file-systems-at-boot-by-universally-unique-id-uuid-or-label-rhcsa-objective-preparation

**Day 5: 2025-11-07 (Friday)**
*   **Topics:** Storage Management - LVM: Create and manage Physical Volumes (PVs), Volume Groups (VGs), and Logical Volumes (LVs), extend and reduce LVM components, mount LVM-based filesystems.
*   **Estimated Time:** Theory: 1.5h, Hands-on: 4h, Review: 1h
*   **Tip:** LVM is a critical RHCSA topic. Practice creating, extending, and reducing LVs with both ext4 and XFS filesystems. Remember to resize the filesystem *after* resizing the logical volume.
*   **Links:**
    *   Tecmint - How to Extend/Reduce LVM's (Logical Volume Management) in Linux: https://www.tecmint.com/extend-and-reduce-lvms-in-linux/
    *   Red Hat Blog - How to resize a logical volume with 5 simple LVM commands: https://www.redhat.com/en/blog/resize-lvm-simple

**Day 6: 2025-11-08 (Saturday)**
*   **Topics:** Networking Configuration & Firewall: Configure IP addresses, subnet masks, and default gateways using `nmcli` or `nmtui`, manage hostname resolution (`hostnamectl`, `/etc/hosts`), configure and manage `firewalld` (zones, services, ports).
*   **Estimated Time:** Theory: 2h, Hands-on: 3.5h, Review: 1h
*   **Tip:** Understand the difference between runtime and permanent `firewalld` changes. Always test changes before making them permanent.
*   **Links:**
    *   Red Hat Docs - Configuring and managing networking: https://docs.redhat.com/en/documentation/red_hat_enterprise_linux/9/html-single/configuring_and_managing_networking/index
    *   Tecmint - How to Configure and Manage Basic Networking in RHEL: https://pro.tecmint.com/setup-network-services-rhel/

**Day 7: 2025-11-09 (Sunday)**
*   **Topics:** Archiving, Compression, & Remote Access: Create and extract archives using `tar`, compress/decompress files (gzip, bzip2, xz), synchronize files with `rsync`, configure and use SSH client and server for remote access (including key-based authentication), transfer files securely with `scp`.
*   **Estimated Time:** Theory: 1.5h, Hands-on: 3h, Review: 1h
*   **Tip:** Practice SSH key-based authentication. It's a common and important skill for system administrators.
*   **Links:**
    *   YouTube - Archiving and Compressing in Red Hat 9: A Complete Guide using Tar, Gzip, Bzip2, XZ, and Zip: https://www.youtube.com/watch?v=5Dim25XJPTc
    *   ADDZEY.DEV - RHCSA Study Notes (Archive, compress, SSH): https://addzey.dev/rhcsa/

**Day 8: 2025-11-10 (Monday)**
*   **Topics:** SELinux & Boot Process Management: Understand and manage SELinux modes and contexts (getenforce, setenforce, ls -Z, chcon, restorecon, getsebool, setsebool), troubleshoot SELinux issues (`audit2allow`), configure GRUB2 bootloader, perform password recovery, and use rescue mode.
*   **Estimated Time:** Theory: 2h, Hands-on: 4h, Review: 1h
*   **Tip:** SELinux can be challenging. Focus on understanding contexts and how to restore them, and how to use `audit2allow` to diagnose issues. Practice root password reset multiple times.
*   **Links:**
    *   Red Hat Docs - Allowing Access: audit2allow: https://docs.redhat.com/en/documentation/red_hat_enterprise_linux/6/html/security-enhanced_linux/sect-security-enhanced_linux-fixing_problems-allowing_access_audit2allow
    *   YouTube - Reset Root Password - RHCSA v9 Review: https://www.youtube.com/watch?v=HK7OI3uboKg

**Day 9: 2025-11-11 (Tuesday)**
*   **Topics:** System Maintenance & Advanced Troubleshooting: Analyze system logs (`journalctl`, `/var/log`), monitor system performance (top, vmstat, iostat), configure NTP clients (`chronyd`), review and practice common troubleshooting scenarios (e.g., network issues, storage issues, boot problems).
*   **Estimated Time:** Theory: 2h, Hands-on: 3.5h, Review: 1h
*   **Tip:** Troubleshooting requires integrating knowledge from all previous days. Create your own scenarios (e.g., break a network configuration, corrupt an fstab entry) and fix them.
*   **Links:**
    *   DigitalOcean - How To Use journalctl to View and Manipulate systemd Logs: https://www.digitalocean.com/community/tutorials/how-to-use-journalctl-to-view-and-manipulate-systemd-logs
    *   Red Hat Docs - Troubleshooting problems by using log files: https://docs.redhat.com/en/documentation/red_hat_enterprise_linux/8/html/configuring_basic_system_settings/assembly_troubleshooting-problems-using-log-files_configuring-basic-system-settings

**Day 10: 2025-11-12 (Wednesday)**
*   **Topics:** Practice Exam & Review: Simulate a full RHCSA practice exam under timed conditions, identify weak areas based on performance, and conduct targeted review of challenging topics.
*   **Estimated Time:** Practice Exam: 3h, Targeted Review: 3h
*   **Tip:** Treat the practice exam like the real thing: no notes, timed, and in a clean environment. This will highlight your actual readiness. Focus your review on the areas where you struggled.
*   **Links:**
    *   PracticeTestGeeks - Free RHCSA Practice Test: https://practicetestgeeks.com/rhcsa-exam/
    *   LabEx.io - RHCSA Certification Exam Practice Exercises: https://labex.io/courses/rhcsa-certification-exam-practice-exercises